In [1]:
import os.path
import pandas as pd

# 读取上传的CSV文件
file_path_compare = './output/merge2/similar_job_list_data_merged.xlsx'
data_compare = pd.read_excel(file_path_compare)
# 显示数据的前几行以了解其结构
data_compare

,id,company_name,similar_job_html,similar_job_name,similar_job_salary,similar_job_company,similar_job_location,similar_job_compare
0,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_detail/af2e598dd5af...,技术部部长/负责人,15-25K·14薪,力扬工业,遂宁,https://www.zhipin.com/job_pk/afa76d124e2b8ec2...
1,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_detail/3290fdbb0df9...,Java项目经理,10-15K,持卡乐,遂宁,https://www.zhipin.com/job_pk/afa76d124e2b8ec2...
2,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_detail/70559450ac2b...,技术研发总监,15-30K,久大盐业,遂宁,https://www.zhipin.com/job_pk/afa76d124e2b8ec2...
3,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_detail/af2e598dd5af...,技术部部长/负责人,15-25K·14薪,力扬工业,遂宁,https://www.zhipin.com/job_pk/afa76d124e2b8ec2...
4,4986,四川厨源不锈钢设备有限公司,https://www.zhipin.com/job_detail/bfeb3b26959a...,授权签字人,6-7K,遂宁市丰平汽车检...,遂宁,https://www.zhipin.com/job_pk/afa76d124e2b8ec2...
...,...,...,...,...,...,...,...,...
14860,8760,苏州中科行智智能科技有限公司,https://www.zhipin.com/job_detail/53a998f3fde1...,销售工程师,8-12K,中正集团,苏州,https://www.zhipin.com/job_pk/f806c8fac656e7aa...
14861,8760,苏州中科行智智能科技有限公司,https://www.zhipin.com/job_detail/c0df808e1c92...,销售工程师,8-13K·15薪,世强,苏州,https://www.zhipin.com/job_pk/f806c8fac656e7aa...
14862,8760,苏州中科行智智能科技有限公司,https://www.zhipin.com/job_detail/c812f8d96d29...,销售工程师（苏州）,8-12K,增广机器人,苏州,https://www.zhipin.com/job_pk/f806c8fac656e7aa...
14863,8760,苏州中科行智智能科技有限公司,https://www.zhipin.com/job_detail/3297cdc94728...,销售工程师,8-13K,江苏康达检测技术...,苏州,https://www.zhipin.com/job_pk/f806c8fac656e7aa...


In [2]:
from MyUtil import Selenium_Edge
from MyUtil import OpenCv_Util
from selenium.webdriver.common.by import By
from pprint import pprint
from time import sleep
edge_driver = Selenium_Edge(False)
# 职位信息
job_original = []
# 对比职位的职位信息
compare_job_data = []
# 对比职位的公司信息
compare_company_data = []
# 加载失败链接
compare_fail_url = []
# 失效链接
compare_error_url = []
# 推荐对比
command_compare_data = []

start_compare = 7169
end_compare = 8900
segment_name_compare = f"data_compare_{start_compare}_{end_compare}"  # 分段命名
segment_data_compare = data_compare[start_compare:end_compare]  # 获取当前分段的数据
segment_data_compare1 = pd.DataFrame(segment_data_compare)

下载目录:
D:\Path\study\Jupyter\crwal


In [3]:
edge_driver.get_driver(data_compare['similar_job_compare'][0])

In [4]:
if 1:
    previous_id = None
    number = start_compare
    for index_compare, data_compare in segment_data_compare1.iterrows():
        url = data_compare['similar_job_compare']
        id = data_compare['id']
        company_name = data_compare['company_name']
        info_id ={
            "number" :number,
            "id":id,
            "company_name_original":company_name
        }
        if url == 'Null':
            compare_error_url.append(info_id)
            continue
        else:
            edge_driver.get_driver(url)
            info = {
                "number":number,
                "url":url,
                "id":id,
                "company_name_original":company_name,
                "compare_error_info":""
            }
            def collect_error():
                if edge_driver.get_element_text('//*[@id="main"]/div/div/div/a') == "首页":
                    info['compare_error_info'] = f"第{number}号对应的{id}的{company_name}相似链接失效"
                    print(info['compare_error_info'])
                    compare_error_url.append(info)
                    return True
                elif not edge_driver.wait_element(xpath='//*[@id="header"]/div[1]/div[2]/ul/li[1]/a',timeout=6,checktext="首页"):
                    info['compare_error_info'] = f"第{number}号对应的{id}的{company_name}相似链接加载失败"
                    print(info['compare_error_info'])
                    compare_fail_url.append(info)
                    return True
                elif edge_driver.get_element_attribute('//*[@id="wrap"]/div[3]/div/div[1]/div[1]/form/div[2]/p/input',attribute='placeholder') == "搜索职位、公司":
                    info['compare_error_info'] = f"第{number}号对应的{id}的{company_name}相似链接失效超时返回主页"
                    print(info['compare_error_info'])
                    compare_error_url.append(info)
                    return True
                else:
                    return False
            def get_company_compare():
                company_father = edge_driver.get_element(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[2]/div[2]')
                abbr_html_compare = edge_driver.get_element_attribute(father_element=company_father, xpath='./div/div/div/div[3]/div/span/a',attribute='href')
                company_name_compare = edge_driver.get_element_text(father_element=company_father, xpath='./div/div/div/div[3]/div/span/a')
                abbr_job_compare = edge_driver.get_element_attribute(father_element=company_father,xpath='./div/div/div/div[2]/h3/a',attribute='href')
                abbr_job_name_compare = edge_driver.get_element_text(father_element=company_father, xpath='./div/div/div/div[2]/h3/a')
                stage_compare = edge_driver.get_element_text(father_element=company_father, xpath="./div/div/div/div[3]/ul/li[2]")
                industry_compare = edge_driver.get_element_text(father_element=company_father, xpath="./div/div/div/div[3]/ul/li[1]")
                edge_driver.click_element('//*[@id="content"]/div/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[2]/h3/a',By.XPATH)
                salary_c = edge_driver.get_element_text(xpath='//*[@id="main"]/div[1]/div/div/div/div[2]/span')
                compare_job_company_compare = {
                        "number":number,
                        "id":id,
                        "compare_company_name":company_name_compare,
                        "compare_abbr_html":abbr_html_compare,
                        "compare_job_html":abbr_job_compare,
                        "compare_abbr_job_name":abbr_job_name_compare,
                        "compare_company_stage":stage_compare,
                        "compare_job_salary":salary_c,
                        "compare_company_industry":industry_compare
                    }
                pprint(compare_job_company_compare)
                if company_father == 'Null':
                    print(f"第{number}号对应的{id}公司简介为空！")
                else:
                    compare_company_data.append(compare_job_company_compare)
                edge_driver.get_driver(url)
                print(f"第{number}号对应的{id}的{company_name}职位信息爬取完毕")

            def get_job_introduce_compare():
                # 职位相关字段
                job_city = job_experience = job_degree = job_tags = job_keyword = job_requirements = job_benefits = company_name_compare = job_time_compare = ""
                company_name_compare = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div[3]/div/span')
                job_father = edge_driver.get_element(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[1]/text()
                job_city = edge_driver.get_element_text(father_element=job_father, xpath='./ul[1]/li[1]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[2]/text()
                job_experience = edge_driver.get_element_text(father_element=job_father, xpath='./ul[1]/li[2]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[3]/text()
                job_degree = edge_driver.get_element_text(father_element=job_father, xpath='./ul[1]/li[3]')
                job_tags = edge_driver.get_element_text(father_element=job_father, xpath='./ul[2]/li')
                job_keyword = edge_driver.get_element_text(father_element=job_father, xpath='./p')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[4]/div[2]/div/ul[1]
                job_requirements = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[4]/div[2]/div/ul[1]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[7]/div[2]/div/ul/li
                job_benefits = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[7]/div[2]/div/ul/li')
                job_time_compare = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[6]/div[2]/div/div/div')
                job_data_compare = {
                    "number":number,
                    "id":id,
                    "compare_company_name":company_name_compare,
                    "compare_job_city":job_city,
                    "compare_job_experience":job_experience,
                    "command_job_degree":job_degree,
                    "command_job_tags":job_tags,
                    "command_job_keyword":job_keyword,
                    "command_job_requirements":job_requirements,
                    "command_job_time":job_time_compare,
                    "command_job_benefits":job_benefits
                }
                pprint(job_data_compare)
                if job_father == 'Null':
                    print(f"第{number}号对应的{id}的{company_name}简介为空！")
                else:
                    compare_job_data.append(job_data_compare)
                print(f"第{number}号对应的{id}的{company_name}职业信息已爬取完毕！")
            def get_recommand_compare():
                recommand_data = []
                url_father = edge_driver.get_elements("//div[@class='recommend']/div[2]")
                for i in url_father:
                    for j in range(1, 9):
                        recommand_name = edge_driver.get_element_attribute(xpath=f"./a[{j}]", father_element=i, attribute='href')
                        recommand_url = edge_driver.get_element_attribute(xpath=f"./a[{j}]", father_element=i)
                        recommand_data.append((recommand_name, recommand_url))
                for recommand in recommand_data:
                    command_compare = {
                        'number':number,
                        'id' : id,
                        'original_company' : company_name,
                        'command_compare_url' : recommand[0],
                        'command_compare_name' : recommand[1]
                    }
                    if recommand != 'Null':
                        command_compare_data.append(command_compare)
                print(f"第{number}号对应的{id}的{company_name}推荐数据爬取成功！")
            def get_original_job():
                # 职位相关字段
                job_city = job_experience = job_degree = job_tags = job_keyword = job_requirements = job_benefits = job_time = ""
                job_father = edge_driver.get_element(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[1]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[1]/div/ul[1]/li[1]
                job_city = edge_driver.get_element_text(father_element=job_father, xpath='./div/ul[1]/li[1]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[2]/text()
                job_experience = edge_driver.get_element_text(father_element=job_father, xpath='./div/ul[1]/li[2]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[1]/li[3]/text()
                job_degree = edge_driver.get_element_text(father_element=job_father, xpath='./div/ul[1]/li[3]')
                # //*[@id="content"]/div/div/div[1]/div[1]/div/div[3]/div[2]/div/ul[2]/li
                job_tags = edge_driver.get_element_text(father_element=job_father, xpath='./div/ul[2]/li')
                job_keyword = edge_driver.get_element_text(father_element=job_father, xpath='./div/p')
                job_requirements = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[4]/div[1]/div/ul[1]')
                job_time = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[6]/div[1]/div/div/div')
                job_benefits = edge_driver.get_element_text(xpath='//*[@id="content"]/div/div/div[1]/div[1]/div/div[7]/div[1]/div/ul')
                original_job_data = {
                    "number":number,
                    "id":id,
                    "original_company_name_c":company_name,
                    "original_job_city":job_city,
                    "original_job_experience":job_experience,
                    "original_job_degree":job_degree,
                    "original_job_tags":job_tags,
                    "original_job_keyword":job_keyword,
                    "original_job_requirements":job_requirements,
                    "original_job_time":job_time,
                    "original_job_benefits":job_benefits
                }
                pprint(original_job_data)
                if job_father != 'Null':
                    job_original.append(original_job_data)
                else:
                    compare_fail_url.append(info)
                    print(f"第{id}号的{company_name}职业信息加载失败！")
            if collect_error():
                continue
            elif id != previous_id:
                get_original_job()
            get_company_compare()
            get_job_introduce_compare()
            get_recommand_compare()
            previous_id = id
            number+=1

{'id': 6684,
 'number': 7169,
 'original_company_name_c': '湖北海震科创技术有限公司',
 'original_job_benefits': '五险一金\n年终奖\n节日福利',
 'original_job_city': '武汉',
 'original_job_degree': '硕士',
 'original_job_experience': '1-3年',
 'original_job_keyword': '一、职位描述\n'
                         '岗位名称：监测方案设计及现场监测与测试、数据分析工程师，3-4人。\n'
                         '二、工作内容：\n'
                         '主要从事岩爆、地压冲击等动力型灾害微震监测方案设计、现场的安装调试、数据的分析与预警、报告编写、软硬件的测试等工作。\n'
                         '三、工作地点：\n'
                         '武汉\n'
                         '四、待遇:\n'
                         '实行“基本工资+差旅补助+岗位业绩奖”薪酬制。\n'
                         '五、招聘条件：\n'
                         '1、身体健康，能吃苦耐劳。\n'
                         '2、工作积极、认真、负责，具有团队合作精神。\n'
                         '3、具有强烈的上进心及学习能力；\n'
                         '4、硕士及以上学历，采矿工程、测量工程、工程地质、岩土工程等相关专业；\n'
                         '5、具有微震、声发射工作经验者优先；具有现场工作经验者优先； \n'
                         '6、能熟练使用office办公软件、Autocad软件等相关软件\n'
                         '七、应聘提供的材料\n'
 

KeyboardInterrupt: 

In [5]:
import os
from MyUtil import OpenCv_Util
true_end = 7474
segment_name = f"{start_compare}_{true_end}"  # 分段命名
df_job_original_dir = f"./compare_output/job_original"
df_fail_url_dir =  f"./compare_output/compare_fail_url"
df_error_url_dir = f"./compare_output/compare_error_url"
df_job_data_compare_dir =  f"./compare_output/compare_job_data"
df_company_data_compare_dir =  f"./compare_output/compare_company_data"
df_job_list_data_compare_dir =  f"./compare_output/command_compare_data"
OpenCv_Util.mkdir(df_job_original_dir)
OpenCv_Util.mkdir(df_fail_url_dir)
OpenCv_Util.mkdir(df_error_url_dir)
OpenCv_Util.mkdir(df_job_data_compare_dir)
OpenCv_Util.mkdir(df_company_data_compare_dir)
OpenCv_Util.mkdir(df_job_list_data_compare_dir)

./compare_output/job_original文件夹已存在
./compare_output/compare_fail_url文件夹已存在
./compare_output/compare_error_url文件夹已存在
./compare_output/compare_job_data文件夹已存在
./compare_output/compare_company_data文件夹已存在
./compare_output/command_compare_data文件夹已存在


In [6]:
# 职位信息
df_job_original_data = pd.DataFrame(job_original)
print(df_job_original_data)
df_job_original_data.to_excel(f'{df_job_original_dir}/{segment_name}.xlsx', index=False)

    number    id original_company_name_c original_job_city  \
0     7169  6684            湖北海震科创技术有限公司                武汉   
1     7173  6685          四川睿跃土木工程咨询有限公司                成都   
2     7178  6686         国家林业和草原局林草调查规划院                西安   
3     7183  6687          贵州勘设生态环境科技有限公司                贵阳   
4     7188  6688            贵州航天电子科技有限公司                贵阳   
5     7191  6689          天津艾睿信佳科技发展有限公司                天津   
6     7196  6691            汉威科技集团股份有限公司                郑州   
7     7201  6693          南京中钞长城金融设备有限公司                南京   
8     7206  6694           武汉德智自动化科技有限公司                武汉   
9     7211  6696            广州山霖生态科技有限公司                广州   
10    7216  6697              沈阳乐节科技有限公司                沈阳   
11    7221  6698          业聚医疗器械（深圳）有限公司                深圳   
12    7226  6699            上海骐杰碳素材料有限公司                上海   
13    7231  6700          武汉中能华源设计咨询有限公司                武汉   
14    7235  6701          山东东润仪表科技股份有限公司                烟台   
15    72

In [7]:
# 对比职位的职位信息
df_look_job_list_data = pd.DataFrame(compare_job_data)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_job_data_compare_dir}/{segment_name}.xlsx', index=False)

     number    id compare_company_name compare_job_city  \
0      7169  6684                 泛微网络               武汉   
1      7170  6684                 三维天地               武汉   
2      7171  6684                 三维天地               武汉   
3      7172  6684                 四格互联               武汉   
4      7173  6685                   江南               成都   
..      ...   ...                  ...              ...   
201    7476  6755             某知名新能源公司               苏州   
202    7477  6755             某知名新能源公司               苏州   
203    7478  6755          某知名仪器仪表上市公司               苏州   
204    7479  6755               汇川联合动力               苏州   
205    7480  6755             某知名新能源公司               苏州   

    compare_job_experience command_job_degree command_job_tags  \
0                     3-5年                 本科              ERP   
1                     1-3年                 本科            可适应出差   
2                     1-3年                 本科            可适应出差   
3                     3-5年 

In [8]:
# 对比职位的公司信息
df_look_job_list_data = pd.DataFrame(compare_company_data)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_company_data_compare_dir}/{segment_name}.xlsx', index=False)

     number    id compare_company_name  \
0      7169  6684                 泛微网络   
1      7170  6684                 三维天地   
2      7171  6684                 三维天地   
3      7172  6684                 四格互联   
4      7173  6685                   江南   
..      ...   ...                  ...   
201    7476  6755                 Null   
202    7477  6755                 Null   
203    7478  6755                 Null   
204    7479  6755               汇川联合动力   
205    7480  6755                 Null   

                                     compare_abbr_html  \
0    https://www.zhipin.com/gongsi/8d038826246b5fb3...   
1    https://www.zhipin.com/gongsi/d2b3b58e76f18a72...   
2    https://www.zhipin.com/gongsi/d2b3b58e76f18a72...   
3    https://www.zhipin.com/gongsi/cc65fecd9ba2d265...   
4    https://www.zhipin.com/gongsi/e7324daabf2ad142...   
..                                                 ...   
201                                               Null   
202                            

In [9]:
# 加载失败链接
df_look_job_list_data = pd.DataFrame(compare_fail_url)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_fail_url_dir}/{segment_name}.xlsx', index=False)

    number                                                url    id  \
0     7245  https://www.zhipin.com/job_pk/afd8c4fdfb4a5a36...  6703   
1     7271  https://www.zhipin.com/job_pk/54fb1a8e6d0667a5...  6710   
2     7276  https://www.zhipin.com/job_pk/dec43ab3613fff9c...  6712   
3     7281  https://www.zhipin.com/job_pk/be29852ac7ac9ebf...  6713   
4     7286  https://www.zhipin.com/job_pk/a52ebc1bd1b5faff...  6714   
5     7291  https://www.zhipin.com/job_pk/475ffd25887c76e0...  6715   
6     7296  https://www.zhipin.com/job_pk/cc0543d5a2f09c3a...  6716   
7     7301  https://www.zhipin.com/job_pk/da4d5c8474210c7d...  6717   
8     7306  https://www.zhipin.com/job_pk/4f5ee650c5de0ef1...  6718   
9     7311  https://www.zhipin.com/job_pk/79d415dc7da2c2d1...  6719   
10    7316  https://www.zhipin.com/job_pk/3bc7b19ea0918883...  6720   
11    7321  https://www.zhipin.com/job_pk/c484891fdb3d521c...  6721   
12    7326  https://www.zhipin.com/job_pk/4d8e126135d32b3b...  6722   
13    

In [10]:
# 失效链接
df_look_job_list_data = pd.DataFrame(compare_error_url)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_error_url_dir}/{segment_name}.xlsx', index=False)

   number                                                url    id  \
0    7231  https://www.zhipin.com/job_pk/43b42e83a75e8f5e...  6700   

  company_name_original                        compare_error_info  
0        武汉中能华源设计咨询有限公司  第7231号对应的6700的武汉中能华源设计咨询有限公司相似链接失效超时返回主页  


In [11]:
# 推荐对比
df_look_job_list_data = pd.DataFrame(command_compare_data)
print(df_look_job_list_data)
df_look_job_list_data.to_excel(f'{df_job_list_data_compare_dir}/{segment_name}.xlsx', index=False)

      number    id original_company  \
0       7169  6684     湖北海震科创技术有限公司   
1       7169  6684     湖北海震科创技术有限公司   
2       7169  6684     湖北海震科创技术有限公司   
3       7169  6684     湖北海震科创技术有限公司   
4       7169  6684     湖北海震科创技术有限公司   
...      ...   ...              ...   
5035    7480  6755       苏州汇川技术有限公司   
5036    7480  6755       苏州汇川技术有限公司   
5037    7480  6755       苏州汇川技术有限公司   
5038    7480  6755       苏州汇川技术有限公司   
5039    7480  6755       苏州汇川技术有限公司   

                                    command_compare_url  \
0     https://www.zhipin.com/job_pk/bbfe1f7ccdac306c...   
1     https://www.zhipin.com/job_pk/f15d986899d7c5a4...   
2     https://www.zhipin.com/job_pk/8b02f41a2db14ad2...   
3     https://www.zhipin.com/job_pk/a45c1ba75ff0264a...   
4     https://www.zhipin.com/job_pk/49ac5c66005d511d...   
...                                                 ...   
5035  https://www.zhipin.com/job_pk/9aa387222597a0ec...   
5036  https://www.zhipin.com/job_pk/eb5a1838e60c8611...   


In [12]:
import pandas as pd
from MyUtil import OpenCv_Util
import os

# 定义一个函数来合并文件夹下的所有Excel文件
def merge_excel_files(folder_path):
    # 初始化一个空的DataFrame用于存储合并后的数据
    merged_df = pd.DataFrame()
    # 遍历文件夹中的所有文件
    for file in os.listdir(folder_path):
        # 检查文件扩展名是否为.xlsx（Excel文件）
        if file.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file)  # 获取文件完整路径
            df = pd.read_excel(file_path)  # 读取Excel文件
            merged_df = pd.concat([merged_df, df], ignore_index=True)  # 合并数据
    # 按照序号排序
    merged_df.sort_values(by='id',inplace=True)
    return merged_df
df_job_original_dir = f"./compare_output/job_original"
df_fail_url_dir =  f"./compare_output/compare_fail_url"
df_error_url_dir = f"./compare_output/compare_error_url"
df_job_data_compare_dir =  f"./compare_output/compare_job_data"
df_company_data_compare_dir =  f"./compare_output/compare_company_data"
df_job_list_data_compare_dir =  f"./compare_output/command_compare_data"
df_dirs = []
df_dirs.append(df_job_original_dir)
df_dirs.append(df_fail_url_dir)
df_dirs.append(df_error_url_dir)
df_dirs.append(df_job_data_compare_dir)
df_dirs.append(df_company_data_compare_dir)
df_dirs.append(df_job_list_data_compare_dir)
# 创建一个新文件夹用于存储合并后的Excel文件
OpenCv_Util.mkdir('./compare_output/merge')
for df_dir in df_dirs:
    data = merge_excel_files(df_dir)
    df_dir = df_dir.replace('./compare_output/','')
    data.to_excel(f'./compare_output/merge/{df_dir}_merged.xlsx', index=False)
    print(df_dir,'处理完毕')

./compare_output/merge文件夹已存在
job_original 处理完毕
compare_fail_url 处理完毕
compare_error_url 处理完毕
compare_job_data 处理完毕
compare_company_data 处理完毕
command_compare_data 处理完毕


In [1]:
import openpyxl
import pandas as pd
from MyUtil import OpenCv_Util

OpenCv_Util.mkdir('./command_compare_output/merge2')
def delete_empty_rows(old_path,save_path):
    # 加载Excel文件
    workbook = openpyxl.load_workbook(old_path)
    sheet = workbook.active
    # 逆序遍历每一行
    for row in reversed(list(sheet.iter_rows(min_row=3))):
        # 检查从第三列开始的数据是否为空
        if all(cell.value == "Null" for cell in row[2:]):
            # 删除该行
            sheet.delete_rows(row[0].row)
    # 保存修改后的Excel文件
    workbook.save(save_path)
file_paths = [
    ('./command_compare_output/merge/command_compare_data_merged.xlsx','./command_compare_output/merge2/command_compare_data_merged.xlsx'),
    ('./command_compare_output/merge/compare_company_data_merged.xlsx','./command_compare_output/merge2/compare_company_data_merged.xlsx'),
    ('./command_compare_output/merge/compare_error_url_merged.xlsx','./command_compare_output/merge2/compare_error_url_merged.xlsx'),
    ('./command_compare_output/merge/compare_fail_url_merged.xlsx','./command_compare_output/merge2/compare_fail_url_merged.xlsx'),
    ('./command_compare_output/merge/compare_job_data_merged.xlsx','./command_compare_output/merge2/compare_job_data_merged.xlsx'),
    ('./command_compare_output/merge/job_original_merged.xlsx','./command_compare_output/merge2/job_original_merged.xlsx')
   # ('./compare_output/merge/omnibus_job_list_data_merged.xlsx','./compare_output/merge2/omnibus_job_list_data_merged.xlsx')
]
for old_path,save_path in file_paths:
    delete_empty_rows(old_path,save_path)
print("处理完毕！")

./command_compare_output/merge2文件夹已创建
处理完毕！


In [3]:
import pandas as pd
from MyUtil import OpenCv_Util
def remove_duplicate_rows(file_path, save_path):
    # 读取Excel文件
    df = pd.read_excel(file_path)
    # 删除重复行数据，保留唯一的行数据
    df_unique = df.drop_duplicates()
    # 保存修改后的Excel文件
    df_unique.to_excel(save_path, index=False)
# 文件路径
OpenCv_Util.mkdir('./command_compare_output/merge3')
file_paths_delete = [
    ('./command_compare_output/merge2/command_compare_data_merged.xlsx','./command_compare_output/merge3/command_compare_data_merged.xlsx'),
    ('./command_compare_output/merge2/compare_company_data_merged.xlsx','./command_compare_output/merge3/compare_company_data_merged.xlsx'),
    ('./command_compare_output/merge2/compare_error_url_merged.xlsx','./command_compare_output/merge3/compare_error_url_merged.xlsx'),
    ('./command_compare_output/merge2/compare_fail_url_merged.xlsx','./command_compare_output/merge3/compare_fail_url_merged.xlsx'),
    ('./command_compare_output/merge2/compare_job_data_merged.xlsx','./command_compare_output/merge3/compare_job_data_merged.xlsx'),
    ('./command_compare_output/merge2/job_original_merged.xlsx','./command_compare_output/merge3/job_original_merged.xlsx'),
]
for file_path,save_path_delete in file_paths_delete:
# 调用函数删除重复行数据
    remove_duplicate_rows(file_path, save_path_delete)
    print(f"{file_path}处理完毕")
print("处理完成！")

./command_compare_output/merge3文件夹已创建
./command_compare_output/merge2/command_compare_data_merged.xlsx处理完毕
./command_compare_output/merge2/compare_company_data_merged.xlsx处理完毕
./command_compare_output/merge2/compare_error_url_merged.xlsx处理完毕
./command_compare_output/merge2/compare_fail_url_merged.xlsx处理完毕
./command_compare_output/merge2/compare_job_data_merged.xlsx处理完毕
./command_compare_output/merge2/job_original_merged.xlsx处理完毕
处理完成！


In [5]:
import openpyxl

def remove_duplicates(input_file, output_file):
    # Load the input workbook
    wb = openpyxl.load_workbook(input_file)
    sheet = wb.active

    # Find unique pairs of values in columns D and E
    unique_pairs = set((row[3], row[4]) for row in sheet.iter_rows(min_row=2, values_only=True))

    # Create a new workbook with only one row for each unique pair
    output_wb = openpyxl.Workbook()
    output_sheet = output_wb.active

    # Copy the header row
    header_row = [cell.value for cell in sheet[1]]
    output_sheet.append(header_row)

    for pair in unique_pairs:
        matching_rows = [row for row in sheet.iter_rows(min_row=2, values_only=True) if row[3:5] == pair]
        output_sheet.append(matching_rows[0])
    # Save the new workbook to the output file
    output_wb.save(output_file)

# Example usage
input_file = './command_compare_output/merge2/command_compare_data_merged.xlsx'
output_file = './command_compare_output/merge3/command_compare_data_merged.xlsx'
remove_duplicates(input_file, output_file)
print("代码运行成功")

代码运行成功


In [ ]:
import openpyxl

def remove_duplicates(input_file, output_file):
    # Load the input workbook
    wb = openpyxl.load_workbook(input_file)
    sheet = wb.active

    # Find unique pairs of values in columns D and H
    unique_pairs = set((row[3], row[7]) for row in sheet.iter_rows(min_row=4, values_only=True))

    # Create a new workbook with only one row for each unique pair
    output_wb = openpyxl.Workbook()
    output_sheet = output_wb.active

    # Copy the header row
    header_row = [cell.value for cell in sheet[1]]
    output_sheet.append(header_row)

    for pair in unique_pairs:
        matching_rows = [row for row in sheet.iter_rows(min_row=4, values_only=True) if row[3:7] == pair]
        output_sheet.append(matching_rows[0])
    # Save the new workbook to the output file
    output_wb.save(output_file)

# Example usage
input_file = './compare_output/merge2/compare_company_data_merged.xlsx'
output_file = './compare_output/merge3/compare_company_data_merged.xlsx'
remove_duplicates(input_file, output_file)
print("代码运行成功")

In [13]:
import pandas as pd
import os

def delete_rows_with_empty_third_column(file_path):
    df = pd.read_excel(file_path)
    df.dropna(subset=[df.columns[2]], inplace=True)
    df.to_excel(file_path, index=False)

def delete_empty_rows_in_xlsx_files(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith('.xlsx'):
            file_path = os.path.join(folder_path, file)
            delete_rows_with_empty_third_column(file_path)
folder_path = "./compare_output/merge/"
delete_empty_rows_in_xlsx_files(folder_path)
print("删除完毕！")

IndexError: index 2 is out of bounds for axis 0 with size 0